<a href="https://colab.research.google.com/github/ishancoderr/Rainfall_predictor/blob/main/Make_predictor_with_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing the libraries**

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import io
import requests

**Importing Rainfall data** 

In [12]:
url='https://raw.githubusercontent.com/ishancoderr/Rainfall_predictor/main/rainfall_data.csv'
s=requests.get(url).content
dataset=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [13]:
dataset.head()

,Unnamed: 0,Avg_Accumulation (mm),Duration (hr),Avg_intensity (mm/hr),occur_
0,0,257,6.0,43,1
1,1,268,5.0,53,1
2,2,289,6.0,48,1
3,3,215,4.0,54,1
4,4,149,3.0,50,1


In [14]:
dataset['occur_'].unique()

array([1, 0])

Hence occur_ can use as output value

In [16]:
dataset = dataset.to_numpy()

In [20]:
print(dataset.shape)

(50, 5)


In [22]:
print(dataset[:5])

[[  0. 257.   6.  43.   1.]
 [  1. 268.   5.  53.   1.]
 [  2. 289.   6.  48.   1.]
 [  3. 215.   4.  54.   1.]
 [  4. 149.   3.  50.   1.]]


In [23]:
X=dataset[:,:-1]

In [25]:
Y=dataset[:,-1]

In [26]:
print(X.shape,Y.shape)

(50, 4) (50,)


In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [83]:
model=Sequential()


model.add(Dense(1,input_dim=len(X[0,:]),activation='sigmoid'))

In here use sigmoid funstion for classification

In [30]:
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [35]:
model.fit(x=X,y=Y,epochs=256,verbose=1)

Epoch 1/256
2/2 [==============================] - 0s 5ms/step - loss: 5.2619 - accuracy: 0.6600
Epoch 2/256
2/2 [==============================] - 0s 3ms/step - loss: 5.2619 - accuracy: 0.6600
Epoch 3/256
2/2 [==============================] - 0s 3ms/step - loss: 5.2619 - accuracy: 0.6600
Epoch 4/256
2/2 [==============================] - 0s 9ms/step - loss: 5.2619 - accuracy: 0.6600
Epoch 5/256
2/2 [==============================] - 0s 3ms/step - loss: 5.2619 - accuracy: 0.6600
Epoch 6/256
2/2 [==============================] - 0s 9ms/step - loss: 5.2619 - accuracy: 0.6600
Epoch 7/256
2/2 [==============================] - 0s 3ms/step - loss: 5.2619 - accuracy: 0.6600
Epoch 8/256
2/2 [==============================] - 0s 6ms/step - loss: 5.2619 - accuracy: 0.6600
Epoch 9/256
2/2 [==============================] - 0s 5ms/step - loss: 5.2619 - accuracy: 0.6600
Epoch 10/256
2/2 [==============================] - 0s 9ms/step - loss: 5.2619 - accuracy: 0.6600
Epoch 11/256
2/2 [===========

In [49]:
print(Y[:45])

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1.]


In [50]:
predictions=model.predict(Y)

In [52]:
print(predictions[:45].T)

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1.]]


In [77]:
model.save('/content/predtest.h5')

In [84]:
TF_LITE_MODEL_FILE_NAME = "tf_lite_model.tflite"

In [85]:
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tf_lite_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpy_b4ahq8/assets


In [91]:
tflite_model_name = TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

852

In [92]:
interpreter = tf.lite.Interpreter(model_path = TF_LITE_MODEL_FILE_NAME)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [1 4]
Input Type: <class 'numpy.float32'>
Output Shape: [1 1]
Output Type: <class 'numpy.float32'>
